In [305]:
import pandas as pd
import numpy as np

In [306]:
# load crowdsourcing dataset from tsv file
df = pd.read_csv('../data/crowdsource/crowd_data.tsv', sep='\t', encoding='utf-8')

In [307]:
df

HITId HITTypeId                                 Title Reward  \
0        1       7QT  Is this triple correct or incorrect?  $0.50   
1        1       7QT  Is this triple correct or incorrect?  $0.50   
2        1       7QT  Is this triple correct or incorrect?  $0.50   
3        1       7QT  Is this triple correct or incorrect?  $0.50   
4        1       7QT  Is this triple correct or incorrect?  $0.50   
..     ...       ...                                   ...    ...   
300     61       9QT  Is this triple correct or incorrect?  $0.50   
301     61       9QT  Is this triple correct or incorrect?  $0.50   
302     61       9QT  Is this triple correct or incorrect?  $0.50   
303     61       9QT  Is this triple correct or incorrect?  $0.50   
304     61       9QT  Is this triple correct or incorrect?  $0.50   

     AssignmentId     WorkerId AssignmentStatus  WorkTimeInSeconds  \
0               1  2133ICYWE97        Submitted                 60   
1               2  2133U7HKDLO        Submitted                 40   
2               3  928UJANWZ12        Submitted                 50   
3               4    1726JMZQW        Submitted                 80   
4               5  2134U7HKDMM        Submitted                  2   
..            ...          ...              ...                ...   
300           301    AALKMII97        Submitted                240   
301           302    HHCKW1111        Submitted                200   
302           303   GGUI83657S        Submitted                120   
303           304  WWHL098SA43        Submitted                 10   
304           305  QZAHIFT8263        Submitted                  2   

    LifetimeApprovalRate     Input1ID   Input2ID   Input3ID  AnswerID  \
0                    99%    wd:Q11621  wdt:P2142  792910554       1.0   
1                    40%    wd:Q11621  wdt:P2142  792910554       1.0   
2                    98%    wd:Q11621  wdt:P2142  792910554       2.0   
3                    70%    wd:Q11621  wdt:P2142  792910554       1.0   
4                    70%    wd:Q11621  wdt:P2142  792910554       1.0   
..                   ...          ...        ...        ...       ...   
300                  98%  wd:Q1288004  wdt:P1412  wd:Q13330       2.0   
301                  80%  wd:Q1288004  wdt:P1412  wd:Q13330       1.0   
302                  85%  wd:Q1288004  wdt:P1412  wd:Q13330       1.0   
303                  69%  wd:Q1288004  wdt:P1412  wd:Q13330       2.0   
304                  40%  wd:Q1288004  wdt:P1412  wd:Q13330       2.0   

    AnswerLabel FixPosition FixValue  
0       CORRECT         NaN      NaN  
1       CORRECT         yes      yes  
2     INCORRECT         NaN      NaN  
3       CORRECT         NaN      NaN  
4       CORRECT         NaN      NaN  
..          ...         ...      ...  
300   INCORRECT      Object    Q1860  
301     CORRECT         NaN      NaN  
302     CORRECT         NaN      NaN  
303   INCORRECT         NaN      NaN  
304   INCORRECT         100      100  

[305 rows x 16 columns]

In [308]:
# clean answers
for idx, lbl in enumerate(df['FixValue'].values):
    if str(lbl)[:3] == 'wd:':
        print(lbl)
        print(str(lbl)[3:])
        df['FixValue'][idx] = str(lbl)[3:]

wd:Q72
Q72
wd:Q94074
Q94074
wd:Q95073
Q95073


/tmp/ipykernel_8105/3954016013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FixValue'][idx] = str(lbl)[3:]


In [309]:
# find and filter out malicious workers 
malicious_workers = {}
# filter based on inputs
for idx, lbl in enumerate(df['AnswerLabel'].values):
    if lbl == 'INCORRECT':
        if str(df['FixValue'][idx]) in ['Yes', 'I don\'t understand', '100']:
            print(str(df['FixPosition'][idx]))
            print(str(df['FixValue'][idx]))
            # store the malicious worker id and the question id
            if df['WorkerId'][idx] not in malicious_workers:
                malicious_workers[df['WorkerId'][idx]] = {}
                malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
                malicious_workers[df['WorkerId'][idx]]['Task'] = []
                malicious_workers[df['WorkerId'][idx]]['Time'] = 0
            malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
            malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
    else:
        if str(df['FixPosition'][idx]) not in ['NaN', '', 'nan']:
            print(str(df['FixPosition'][idx]))
            print(str(df['FixValue'][idx])) 
            if df['WorkerId'][idx] not in malicious_workers:
                malicious_workers[df['WorkerId'][idx]] = {}
                malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
                malicious_workers[df['WorkerId'][idx]]['Task'] = []
                malicious_workers[df['WorkerId'][idx]]['Time'] = 0
            malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
            malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
# filter based on working time
for idx, time in enumerate(df['WorkTimeInSeconds'].values):
    if time < 20 and str(df['FixValue'][idx]) in ['Yes', 'I don\'t understand', '100', '', 'nan', 'NaN']:
        print(str(df['FixPosition'][idx]))
        print(str(df['FixValue'][idx]))
        if df['WorkerId'][idx] not in malicious_workers:
            malicious_workers[df['WorkerId'][idx]] = {}
            malicious_workers[df['WorkerId'][idx]]['Approval'] = df['LifetimeApprovalRate'][idx]
            malicious_workers[df['WorkerId'][idx]]['Task'] = []
            malicious_workers[df['WorkerId'][idx]]['Time'] = 0
        malicious_workers[df['WorkerId'][idx]]['Task'] = np.append(malicious_workers[df['WorkerId'][idx]]['Task'], df['HITId'][idx])
        malicious_workers[df['WorkerId'][idx]]['Time'] += df['WorkTimeInSeconds'][idx]
# get average work time for each worker
for worker in malicious_workers:
    malicious_workers[worker]['Time'] /= len(malicious_workers[worker]['Task'])


yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
I don't understand
I don't understand
I don't understand
I don't understand
100
100
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
I don't understand
100
100
100
100
100
100
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [310]:
malicious_workers

{'2133U7HKDLO': {'Approval': '40%',
  'Task': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21.]),
  'Time': 41.0},
 'WWHL098SA43': {'Approval': '69%',
  'Task': array([42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
         55., 56., 57., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
         52., 53., 54., 55., 56., 57., 58., 59., 60., 61.]),
  'Time': 10.0},
 'QZAHIFT8263': {'Approval': '40%',
  'Task': array([44., 58., 60., 61., 42., 43., 44., 45., 46., 47., 48., 49., 50.,
         51., 52., 54., 55., 56., 57., 58., 59., 60., 61.]),
  'Time': 2.0869565217391304},
 '2134U7HKDMM': {'Approval': '70%',
  'Task': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
         14., 15., 16., 17., 18., 19., 20., 21.]),
  'Time': 2.0},
 'LPQMUDT6729': {'Approval': '40%',
  'Task': array([22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34.,
         35., 36., 37., 38., 39

In [311]:
# drop malicious workers from the dataset
for worker in malicious_workers:
    df = df[df.WorkerId != worker]

In [312]:
df

HITId HITTypeId                                 Title Reward  \
0        1       7QT  Is this triple correct or incorrect?  $0.50   
2        1       7QT  Is this triple correct or incorrect?  $0.50   
3        1       7QT  Is this triple correct or incorrect?  $0.50   
6        2       7QT  Is this triple correct or incorrect?  $0.50   
7        2       7QT  Is this triple correct or incorrect?  $0.50   
..     ...       ...                                   ...    ...   
296     60       9QT  Is this triple correct or incorrect?  $0.50   
297     60       9QT  Is this triple correct or incorrect?  $0.50   
300     61       9QT  Is this triple correct or incorrect?  $0.50   
301     61       9QT  Is this triple correct or incorrect?  $0.50   
302     61       9QT  Is this triple correct or incorrect?  $0.50   

     AssignmentId     WorkerId AssignmentStatus  WorkTimeInSeconds  \
0               1  2133ICYWE97        Submitted                 60   
2               3  928UJANWZ12        Submitted                 50   
3               4    1726JMZQW        Submitted                 80   
6               7  2133ICYWE97        Submitted                120   
7               8  928UJANWZ12        Submitted                 60   
..            ...          ...              ...                ...   
296           297    HHCKW1111        Submitted                200   
297           298   GGUI83657S        Submitted                120   
300           301    AALKMII97        Submitted                240   
301           302    HHCKW1111        Submitted                200   
302           303   GGUI83657S        Submitted                120   

    LifetimeApprovalRate      Input1ID   Input2ID   Input3ID  AnswerID  \
0                    99%     wd:Q11621  wdt:P2142  792910554       1.0   
2                    98%     wd:Q11621  wdt:P2142  792910554       2.0   
3                    70%     wd:Q11621  wdt:P2142  792910554       1.0   
6                    99%    wd:Q603545  wdt:P2142    4300000       1.0   
7                    98%    wd:Q603545  wdt:P2142    4300000       1.0   
..                   ...           ...        ...        ...       ...   
296                  80%  wd:Q21060270    wdt:P27    wd:Q916       1.0   
297                  85%  wd:Q21060270    wdt:P27    wd:Q916       1.0   
300                  98%   wd:Q1288004  wdt:P1412  wd:Q13330       2.0   
301                  80%   wd:Q1288004  wdt:P1412  wd:Q13330       1.0   
302                  85%   wd:Q1288004  wdt:P1412  wd:Q13330       1.0   

    AnswerLabel FixPosition FixValue  
0       CORRECT         NaN      NaN  
2     INCORRECT         NaN      NaN  
3       CORRECT         NaN      NaN  
6       CORRECT         NaN      NaN  
7       CORRECT         NaN      NaN  
..          ...         ...      ...  
296     CORRECT         NaN      NaN  
297     CORRECT         NaN      NaN  
300   INCORRECT      Object    Q1860  
301     CORRECT         NaN      NaN  
302     CORRECT         NaN      NaN  

[183 rows x 16 columns]

In [316]:
# group by HITId
grouped = df.groupby('HITId')
# save majority vote answers to csv file
crowd_answers = {'HITId': [], 'kappa': [], 'label': [], 'support': [], 'Subject': [], 'Predicate': [], 'Object': [], 'FixPosition': [], 'FixValue': []}
for name, group in grouped:
    # get the majority vote
    majority_vote = group['AnswerLabel'].value_counts().idxmax()
    # check if the majority vote is not agreed upon
    print(name)
    print(group['AnswerLabel'].value_counts())
    print('Majority vote: ' + majority_vote)
    if len(group['FixPosition'].value_counts()) == 1:
        crowd_answers['HITId'] = np.append(crowd_answers['HITId'], name)
        crowd_answers['kappa'] = np.append(crowd_answers['kappa'], crowd_kappa)
        # append number of workers that voted for the majority vote
        crowd_answers['label'] = np.append(crowd_answers['label'], majority_vote)
        crowd_answers['support'] = np.append(crowd_answers['support'], group['AnswerLabel'].value_counts().max())
        crowd_answers['Subject'] = np.append(crowd_answers['Subject'], group['Input1ID'].values[0])
        crowd_answers['Predicate'] = np.append(crowd_answers['Predicate'], group['Input2ID'].values[0])
        crowd_answers['Object'] = np.append(crowd_answers['Object'], group['Input3ID'].values[0])
        crowd_answers['FixPosition'] = np.append(crowd_answers['FixPosition'], group['FixPosition'].value_counts().idxmax())
        print('Fix position: ' + group['FixPosition'].value_counts().idxmax())
    else:
        crowd_answers['HITId'] = np.append(crowd_answers['HITId'], name)
        crowd_answers['kappa'] = np.append(crowd_answers['kappa'], crowd_kappa)
        # append number of workers that voted for the majority vote
        crowd_answers['label'] = np.append(crowd_answers['label'], majority_vote)
        crowd_answers['support'] = np.append(crowd_answers['support'], group['AnswerLabel'].value_counts().max())
        crowd_answers['Subject'] = np.append(crowd_answers['Subject'], group['Input1ID'].values[0])
        crowd_answers['Predicate'] = np.append(crowd_answers['Predicate'], group['Input2ID'].values[0])
        crowd_answers['Object'] = np.append(crowd_answers['Object'], group['Input3ID'].values[0])
        crowd_answers['FixPosition'] = np.append(crowd_answers['FixPosition'], 'NA')
    if len(group['FixValue'].value_counts()) == 1:
        crowd_answers['FixValue'] = np.append(crowd_answers['FixValue'], group['FixValue'].value_counts().idxmax())
        print('Fix value: ' + group['FixValue'].value_counts().idxmax())
    else:
        crowd_answers['FixValue'] = np.append(crowd_answers['FixValue'], 'NA')
    print('')

1
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

2
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

3
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object
Fix value: 2014-02-18

4
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object

5
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

6
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 2019-02-24

7
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object

8
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 698491348

9
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

10
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

11
CORRECT      2


In [317]:
crowd_answers = pd.DataFrame(crowd_answers)
crowd_answers

HITId  kappa      label  support       Subject  Predicate        Object  \
0     1.0   0.17    CORRECT      2.0     wd:Q11621  wdt:P2142     792910554   
1     2.0   0.17    CORRECT      3.0    wd:Q603545  wdt:P2142       4300000   
2     3.0   0.17    CORRECT      2.0  wd:Q16911843   wdt:P577    2014-01-18   
3     4.0   0.17  INCORRECT      3.0    wd:Q132863  wdt:P2142     969023261   
4     5.0   0.17    CORRECT      3.0   wd:Q1628022   wdt:P577    1951-01-01   
..    ...    ...        ...      ...           ...        ...           ...   
56   57.0   0.17    CORRECT      2.0    wd:Q223596  wdt:P1431    wd:Q457180   
57   58.0   0.17  INCORRECT      3.0    wd:Q943992   wdt:P161    wd:Q160432   
58   59.0   0.17    CORRECT      2.0   wd:Q1893555   wdt:P272  wd:Q48784114   
59   60.0   0.17    CORRECT      2.0  wd:Q21060270    wdt:P27       wd:Q916   
60   61.0   0.17    CORRECT      2.0   wd:Q1288004  wdt:P1412     wd:Q13330   

   FixPosition    FixValue  
0           NA          NA  
1           NA          NA  
2       Object  2014-02-18  
3       Object          NA  
4           NA          NA  
..         ...         ...  
56      Object     Q181900  
57          NA          NA  
58      Object    Q7488442  
59      Object        Q884  
60      Object       Q1860  

[61 rows x 9 columns]

In [318]:
# save the majority vote answers to csv file
crowd_answers.to_csv('../data/crowdsource/crowd_answers.csv', index=False)

# Triples to Correct in KG

In [241]:
# majority vote, show results that are not agreed upon
# group by HITId
grouped = df.groupby('HITId')
# iterate over groups
triples_to_correct = {'HITId': [], 'kappa': [], 'support': [], 'Subject': [], 'Predicate': [], 'Object': [], 'FixPosition': [], 'FixValue': []}
for name, group in grouped:
    # get the majority vote
    majority_vote = group['AnswerLabel'].value_counts().idxmax()
    # check if the majority vote is not agreed upon
    print(name)
    print(group['AnswerLabel'].value_counts())
    print('Majority vote: ' + majority_vote)
    if len(group['FixPosition'].value_counts()) == 1:
        if majority_vote == 'INCORRECT':
            triples_to_correct['HITId'] = np.append(triples_to_correct['HITId'], name)
            triples_to_correct['kappa'] = np.append(triples_to_correct['kappa'], crowd_kappa)
            # append number of workers that voted for the majority vote
            triples_to_correct['support'] = np.append(triples_to_correct['support'], group['AnswerLabel'].value_counts().max())
            triples_to_correct['Subject'] = np.append(triples_to_correct['Subject'], group['Input1ID'].values[0])
            triples_to_correct['Predicate'] = np.append(triples_to_correct['Predicate'], group['Input2ID'].values[0])
            triples_to_correct['Object'] = np.append(triples_to_correct['Object'], group['Input3ID'].values[0])
            triples_to_correct['FixPosition'] = np.append(triples_to_correct['FixPosition'], group['FixPosition'].value_counts().idxmax())
        print('Fix position: ' + group['FixPosition'].value_counts().idxmax())
    else:
        if majority_vote == 'INCORRECT':
            triples_to_correct['HITId'] = np.append(triples_to_correct['HITId'], name)
            triples_to_correct['kappa'] = np.append(triples_to_correct['kappa'], crowd_kappa)
            # append number of workers that voted for the majority vote
            triples_to_correct['support'] = np.append(triples_to_correct['support'], group['AnswerLabel'].value_counts().max())
            triples_to_correct['Subject'] = np.append(triples_to_correct['Subject'], group['Input1ID'].values[0])
            triples_to_correct['Predicate'] = np.append(triples_to_correct['Predicate'], group['Input2ID'].values[0])
            triples_to_correct['Object'] = np.append(triples_to_correct['Object'], group['Input3ID'].values[0])
            triples_to_correct['FixPosition'] = np.append(triples_to_correct['FixPosition'], 'NA')
    if len(group['FixValue'].value_counts()) == 1:
        if majority_vote == 'INCORRECT':
            triples_to_correct['FixValue'] = np.append(triples_to_correct['FixValue'], group['FixValue'].value_counts().idxmax())
        print('Fix value: ' + group['FixValue'].value_counts().idxmax())
    else:
        if majority_vote == 'INCORRECT':
            triples_to_correct['FixValue'] = np.append(triples_to_correct['FixValue'], 'NA')
    print('')

1
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

2
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

3
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object
Fix value: 2014-02-18

4
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object

5
CORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT

6
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 2019-02-24

7
CORRECT      2
INCORRECT    1
Name: AnswerLabel, dtype: int64
Majority vote: CORRECT
Fix position: Object

8
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT
Fix position: Object
Fix value: 698491348

9
INCORRECT    3
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

10
INCORRECT    2
CORRECT      1
Name: AnswerLabel, dtype: int64
Majority vote: INCORRECT

11
CORRECT      2


In [242]:
triples_to_correct

{'HITId': array([ 4.,  6.,  8.,  9., 10., 14., 17., 19., 20., 21., 22., 23., 24.,
        26., 27., 29., 32., 34., 35., 36., 38., 40., 41., 42., 43., 44.,
        45., 46., 47., 58.]),
 'kappa': array([0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17,
        0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17,
        0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17, 0.17]),
 'support': array([3., 2., 3., 3., 2., 3., 2., 2., 2., 2., 2., 3., 2., 3., 2., 3., 2.,
        3., 2., 2., 2., 2., 2., 2., 3., 3., 2., 3., 3., 3.]),
 'Subject': array(['wd:Q132863', 'wd:Q48313910', 'wd:Q217010', 'wd:Q4993462',
        'wd:Q7033842', 'wd:Q1004', 'wd:Q16101952', 'wd:Q15715406',
        'wd:Q15239622', 'wd:Q708135', 'wd:Q1720855', 'wd:Q598752',
        'wd:Q427386', 'wd:Q1410031', 'wd:Q20001199', 'wd:Q639070',
        'wd:Q2893887', 'wd:Q1039395', 'wd:Q202211', 'wd:Q28974159',
        'wd:Q885281', 'wd:Q6548158', 'wd:Q1123629', 'wd:Q1780602',
        'wd:Q610633', 'wd:Q2188914', 

In [243]:
triples_to_correct = pd.DataFrame(triples_to_correct)

In [244]:
triples_to_correct

HITId  kappa  support       Subject                Predicate  \
0     4.0   0.17      3.0    wd:Q132863                wdt:P2142   
1     6.0   0.17      2.0  wd:Q48313910                 wdt:P577   
2     8.0   0.17      3.0    wd:Q217010                wdt:P2142   
3     9.0   0.17      3.0   wd:Q4993462                 wdt:P577   
4    10.0   0.17      2.0   wd:Q7033842                 wdt:P577   
5    14.0   0.17      3.0      wd:Q1004  ddis:indirectSubclassOf   
6    17.0   0.17      2.0  wd:Q16101952  ddis:indirectSubclassOf   
7    19.0   0.17      2.0  wd:Q15715406  ddis:indirectSubclassOf   
8    20.0   0.17      2.0  wd:Q15239622  ddis:indirectSubclassOf   
9    21.0   0.17      2.0    wd:Q708135  ddis:indirectSubclassOf   
10   22.0   0.17      2.0   wd:Q1720855                 wdt:P577   
11   23.0   0.17      3.0    wd:Q598752                 wdt:P577   
12   24.0   0.17      2.0    wd:Q427386                wdt:P2142   
13   26.0   0.17      3.0   wd:Q1410031                 wdt:P577   
14   27.0   0.17      2.0  wd:Q20001199                wdt:P2142   
15   29.0   0.17      3.0    wd:Q639070                 wdt:P161   
16   32.0   0.17      2.0   wd:Q2893887                 wdt:P161   
17   34.0   0.17      3.0   wd:Q1039395                 wdt:P161   
18   35.0   0.17      2.0    wd:Q202211                 wdt:P495   
19   36.0   0.17      2.0  wd:Q28974159                 wdt:P750   
20   38.0   0.17      2.0    wd:Q885281                 wdt:P161   
21   40.0   0.17      2.0   wd:Q6548158                 wdt:P136   
22   41.0   0.17      2.0   wd:Q1123629                 wdt:P725   
23   42.0   0.17      2.0   wd:Q1780602                 wdt:P921   
24   43.0   0.17      3.0    wd:Q610633                wdt:P2554   
25   44.0   0.17      3.0   wd:Q2188914                 wdt:P161   
26   45.0   0.17      2.0   wd:Q1032889                  wdt:P58   
27   46.0   0.17      3.0    wd:Q209538                wdt:P3650   
28   47.0   0.17      3.0  wd:Q31202708                wdt:P3092   
29   58.0   0.17      3.0    wd:Q943992                 wdt:P161   

          Object FixPosition    FixValue  
0      969023261      Object          NA  
1     2018-02-24      Object  2019-02-24  
2      698491347      Object   698491348  
3     2008-08-29          NA          NA  
4     1996-06-01          NA          NA  
5   wd:Q47461344          NA          NA  
6      wd:Q95074      Object      Q95073  
7   wd:Q27096213          NA          NA  
8   wd:Q27096213          NA          NA  
9     wd:Q618779          NA          NA  
10    2010-10-01      Object  2010-01-01  
11    2001-01-01      Object  2011-01-01  
12     176997186      Object   176997168  
13    2010-10-01      Object  2011-01-01  
14     863756050      Object   863756051  
15   wd:Q2865083      Object    Q5423258  
16    wd:Q105825     Subject    Q1168152  
17  wd:Q17386547     Subject     Q908556  
18        wd:Q30     Subject    Q1471671  
19    wd:Q907311     Subject    Q1722254  
20   wd:Q3087184     Subject     Q409022  
21     wd:Q33999   Predicate        P106  
22    wd:Q211040   Predicate        P161  
23    wd:Q295093   Predicate        P344  
24   wd:Q1700810   Predicate         P58  
25   wd:Q2416526      Object   Q17350908  
26   wd:Q4762311      Object    Q3194791  
27  wd:Q28732983      Object   Q28732985  
28   wd:Q4805527   Predicate         P58  
29    wd:Q160432          NA          NA

In [ ]:
# save triples to correct to csv
triples_to_correct.to_csv('../data/crowdsource/triples_to_correct.csv', index=False)

In [219]:
# get the inter-rate agreement for each question (HITId)
# inter-rate agreement is calculated by Fleiss' Kappa
# https://en.wikipedia.org/wiki/Fleiss%27_kappa
def inter_rate_agreement(df):
    # get the number of workers who answered the question
    n = len(df['WorkerId'].unique())
    n = 3
    # get the number of labels
    k = len(df['AnswerLabel'].unique())
    # get the number of questions
    N = len(df['HITId'].unique())
    # get the number of labels for each question
    #N = df.groupby('HITId').size().values
    print(n, k, N)
    # get the number of labels for each question and each label
    Nij = np.zeros((N, k))
    for i in range(N):
        for j in range(k):
            Nij[i, j] = len(df[(df['HITId'] == df['HITId'].unique()[i]) & (df['AnswerLabel'] == df['AnswerLabel'].unique()[j])])
    # calculate the inter-rate agreement
    pi = np.sum(Nij * (Nij - 1), axis=1) / (n * (n - 1))
    pj = np.sum(Nij, axis=0) / (N * n)
    print(pi)
    print(pj)
    pbar = np.sum(pi) / N
    pbarE = np.sum(np.square(pj), axis=0)
    kappa = (pbar - pbarE) / (1 - pbarE)
    print(f"kappa = ({pbar} - {pbarE}) / (1 - {pbarE})")
    return kappa

In [224]:
# apply inter-rate agreement to the dataset
crowd_kappa = inter_rate_agreement(df).round(2)
crowd_kappa

3 2 61
[0.33333333 1.         0.33333333 1.         1.         0.33333333
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 1.         1.         0.33333333 0.33333333 1.
 0.33333333 0.33333333 0.33333333 0.33333333 1.         0.33333333
 0.33333333 1.         0.33333333 1.         1.         1.
 0.33333333 0.33333333 0.33333333 1.         0.33333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 1.         1.         0.33333333 1.         1.         0.33333333
 0.33333333 1.         1.         1.         0.33333333 0.33333333
 0.33333333 0.33333333 0.33333333 1.         0.33333333 0.33333333
 0.33333333]
[0.4863388 0.5136612]
kappa = (0.5846994535519127 - 0.5003732568903222) / (1 - 0.5003732568903222)


0.17

# Correct Knowledge Graph

In [265]:
# import triples to correct
triples_to_correct = pd.read_csv('../data/crowdsource/triples_to_correct.csv')

In [266]:
triples_to_correct

HITId  kappa  support    Subject           Predicate      Object  \
0       4   0.17        3    Q132863               P2142   969023261   
1       6   0.17        2  Q48313910                P577  2018-02-24   
2       8   0.17        3    Q217010               P2142   698491347   
3       9   0.17        3   Q4993462                P577  2008-08-29   
4      10   0.17        2   Q7033842                P577  1996-06-01   
5      14   0.17        3      Q1004  indirectSubclassOf   Q47461344   
6      17   0.17        2  Q16101952  indirectSubclassOf      Q95074   
7      19   0.17        2  Q15715406  indirectSubclassOf   Q27096213   
8      20   0.17        2  Q15239622  indirectSubclassOf   Q27096213   
9      21   0.17        2    Q708135  indirectSubclassOf     Q618779   
10     22   0.17        2   Q1720855                P577  2010-10-01   
11     23   0.17        3    Q598752                P577  2001-01-01   
12     24   0.17        2    Q427386               P2142   176997186   
13     26   0.17        3   Q1410031                P577  2010-10-01   
14     27   0.17        2  Q20001199               P2142   863756050   
15     29   0.17        3    Q639070                P161    Q2865083   
16     32   0.17        2   Q2893887                P161     Q105825   
17     34   0.17        3   Q1039395                P161   Q17386547   
18     35   0.17        2    Q202211                P495         Q30   
19     36   0.17        2  Q28974159                P750     Q907311   
20     38   0.17        2    Q885281                P161    Q3087184   
21     40   0.17        2   Q6548158                P136      Q33999   
22     41   0.17        2   Q1123629                P725     Q211040   
23     42   0.17        2   Q1780602                P921     Q295093   
24     43   0.17        3    Q610633               P2554    Q1700810   
25     44   0.17        3   Q2188914                P161    Q2416526   
26     45   0.17        2   Q1032889                 P58    Q4762311   
27     46   0.17        3    Q209538               P3650   Q28732983   
28     47   0.17        3  Q31202708               P3092    Q4805527   
29     58   0.17        3    Q943992                P161     Q160432   

   FixPosition    FixValue  
0       Object         NaN  
1       Object  2019-02-24  
2       Object   698491348  
3          NaN         NaN  
4          NaN         NaN  
5          NaN         NaN  
6       Object      Q95073  
7          NaN         NaN  
8          NaN         NaN  
9          NaN         NaN  
10      Object  2010-01-01  
11      Object  2011-01-01  
12      Object   176997168  
13      Object  2011-01-01  
14      Object   863756051  
15      Object    Q5423258  
16     Subject    Q1168152  
17     Subject     Q908556  
18     Subject    Q1471671  
19     Subject    Q1722254  
20     Subject     Q409022  
21   Predicate        P106  
22   Predicate        P161  
23   Predicate        P344  
24   Predicate         P58  
25      Object   Q17350908  
26      Object    Q3194791  
27      Object   Q28732985  
28   Predicate         P58  
29      Object    Q7360827

In [295]:
# import fixed knowledge graph from pickle
import pickle
with open('../data/fixed_graph.pickle', 'rb') as f:
    fixed_graph = pickle.load(f)

In [296]:
import rdflib
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, XSD

# define some prefixes
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [303]:
len(fixed_graph)

2056778

In [298]:
triples = fixed_graph.triples((None, None, None))

In [300]:
# delete incorrect triples from the fixed knowledge graph
for index, row in triples_to_correct.iterrows():
    if row['Predicate'] == 'indirectSubclassOf':
        print(WD[row['Subject']], DDIS.indirectSubclassOf, WD[(row['Object'])])
        fixed_graph.remove((WD[row['Subject']], DDIS.indirectSubclassOf, WD[(row['Object'])]))
    else:
        print(WD[row['Subject']], WDT[row['Predicate']], WD[(row['Object'])])
        fixed_graph.remove((WD[row['Subject']], WDT[row['Predicate']], WD[(row['Object'])]))

http://www.wikidata.org/entity/Q132863 http://www.wikidata.org/prop/direct/P2142 http://www.wikidata.org/entity/969023261
http://www.wikidata.org/entity/Q48313910 http://www.wikidata.org/prop/direct/P577 http://www.wikidata.org/entity/2018-02-24
http://www.wikidata.org/entity/Q217010 http://www.wikidata.org/prop/direct/P2142 http://www.wikidata.org/entity/698491347
http://www.wikidata.org/entity/Q4993462 http://www.wikidata.org/prop/direct/P577 http://www.wikidata.org/entity/2008-08-29
http://www.wikidata.org/entity/Q7033842 http://www.wikidata.org/prop/direct/P577 http://www.wikidata.org/entity/1996-06-01
http://www.wikidata.org/entity/Q1004 http://ddis.ch/atai/indirectSubclassOf http://www.wikidata.org/entity/Q47461344
http://www.wikidata.org/entity/Q16101952 http://ddis.ch/atai/indirectSubclassOf http://www.wikidata.org/entity/Q95074
http://www.wikidata.org/entity/Q15715406 http://ddis.ch/atai/indirectSubclassOf http://www.wikidata.org/entity/Q27096213
http://www.wikidata.org/entity

In [302]:
# add correct triples to the fixed knowledge graph
for index, row in triples_to_correct.iterrows():
    if str(row['FixValue']) not in ['NA', 'NaN', 'nan']:
        if row['Predicate'] == 'indirectSubclassOf':
            if row['FixPosition'] == 'Subject':
                fixed_graph.add((WD[row['FixValue']], DDIS.indirectSubclassOf, WD[(row['Object'])]))
            elif row['FixPosition'] == 'Object':
                fixed_graph.add((WD[row['Subject']], DDIS.indirectSubclassOf, WD[(row['FixValue'])]))
            elif row['FixPosition'] == 'Predicate':
                fixed_graph.add((WD[row['Subject']], DDIS[row['FixValue']], WD[(row['Object'])]))
            else:
                print("unknown fix position")
        else:
            if row['FixPosition'] == 'Subject':
                fixed_graph.add((WD[row['FixValue']], WDT[row['Predicate']], WD[(row['Object'])]))
            elif row['FixPosition'] == 'Object':
                fixed_graph.add((WD[row['Subject']], WDT[row['Predicate']], WD[(row['FixValue'])]))
            elif row['FixPosition'] == 'Predicate':
                fixed_graph.add((WD[row['Subject']], WDT[row['FixValue']], WD[(row['Object'])]))
            else:
                print("unknown fix position")
    else:
        print("no fix value")

no fix value
no fix value
no fix value
no fix value
no fix value
no fix value
no fix value


In [304]:
# save fixed knowledge graph to pickle
with open('../data/fixed_graph_crowd.pickle', 'wb') as f:
    pickle.dump(fixed_graph, f)